In [1]:
# from flair.models import TextClassifier
# from flair.data import Sentence
# classifier = TextClassifier.load('en-sentiment')
# sentence = Sentence('Flair is pretty neat!')
# classifier.predict(sentence)
# # print sentence with predicted labels
# print('Sentence above is: ', sentence.labels)

In [2]:
import pandas as pd
import numpy as np
import itertools
import codecs
import os

In [3]:
# get the current working directory
data_path = os.getcwd()+'/Datasets/'
# os.path.abspath(os.path.join(os.path.dirname( '__file__' ), '..', 'Datasets'))+'/'

# Although the dataset says csv, it is tab delimited. In addition to this, they have severe codels problems. 
# So best to parse throught codes first. 
# UnicodeDecodeError: 'utf-8' codec can't decode byte 0xfc in position 2: invalid start byte

#open for reading with "universal" type set

import codecs

doc_d_t = codecs.open(data_path+'EUADR_Corpus_IBIgroup/'+'EUADR_drug_target'+'.csv','rU','UTF-8') 
EUADR_drug_target = pd.read_csv(doc_d_t, sep='\t', na_filter = False)
EUADR_drug_target['CLASS'] = 'drug_gene'

doc_t_d = codecs.open(data_path+'EUADR_Corpus_IBIgroup/'+'EUADR_target_disease'+'.csv','rU','UTF-8',errors='ignore') 
EUADR_target_disease = pd.read_csv(doc_t_d, sep='\t', na_filter = False)
EUADR_target_disease['CLASS'] = 'gene_disease'
       
doc_d_d = codecs.open(data_path+'EUADR_Corpus_IBIgroup/'+'EUADR_drug_disease'+'.csv','rU','UTF-8')                       
EUADR_drug_disease = pd.read_csv(doc_d_d, sep='\t', na_filter = False)
EUADR_drug_disease['CLASS'] = 'drug_disease'


EUADR_temp = EUADR_drug_target.append(EUADR_target_disease).append(EUADR_drug_disease)

EUADR_temp_1 = EUADR_temp[EUADR_temp['ASSOCIATION_TYPE'] == 'PA']
EUADR_temp_2 = EUADR_temp[EUADR_temp['ASSOCIATION_TYPE'] == 'NA']
EUADR_temp = EUADR_temp_1.append(EUADR_temp_2)
EUADR_temp.rename(columns={"SENTENCE": "text", "ASSOCIATION_TYPE": "label"}, inplace=True)

EUADR_temp.reset_index(inplace=True)

In [4]:
# Get GAD dataset

doc_t_d = codecs.open(data_path+'GAD_Corpus_IBIgroup/'+'GAD_Y_N'+'.csv','rU','UTF-8',errors='ignore') 
GAD_target_disease_Y_N = pd.read_csv(doc_t_d, sep='\t', na_filter = False)
GAD_target_disease_Y_N['CLASS'] = 'gene_disease'

doc_t_d = codecs.open(data_path+'GAD_Corpus_IBIgroup/'+'GAD_F'+'.csv','rU','UTF-8',errors='ignore') 
GAD_target_disease_F = pd.read_csv(doc_t_d, sep='\t', na_filter = False)
GAD_target_disease_F['CLASS'] = 'gene_disease'

GAD_temp = GAD_target_disease_Y_N



GAD_temp['GAD_ASSOC'] = GAD_temp['GAD_ASSOC'].apply(lambda x: x.replace('Y', 'PA').replace('N', 'NA').replace('F', 'FA'))

GAD_temp.rename(columns={"GAD_ASSOC": "label", "GAD_CONCLUSION": "text"}, inplace=True)
GAD_temp.reset_index(inplace=True)

In [5]:
df = EUADR_temp[['label','text']].append(GAD_temp[['label','text']])
df.reset_index(inplace=True)

In [6]:
df.drop("index", axis=1, inplace=True)


In [9]:
data_loc = data_path+'EU-ADR+GAD/'

data = df
data['label'] = '__label__' + data['label'].astype(str)
data.iloc[0:int(len(data)*0.8)].to_csv(data_loc+'train.csv', sep='\t', index = False, header = False)
data.iloc[int(len(data)*0.8):int(len(data)*0.9)].to_csv(data_loc+'test.csv', sep='\t', index = False, header = False)
data.iloc[int(len(data)*0.9):].to_csv(data_loc+'dev.csv', sep='\t', index = False, header = False);


In [11]:
from flair.data_fetcher import NLPTaskDataFetcher
from flair.embeddings import WordEmbeddings, FlairEmbeddings, DocumentRNNEmbeddings
from flair.models import TextClassifier
from flair.trainers import ModelTrainer
from pathlib import Path



In [12]:
corpus = NLPTaskDataFetcher.load_classification_corpus(Path(data_loc), test_file='test.csv', dev_file='dev.csv', train_file='train.csv')


2019-09-10 14:10:45,485 Reading data from /nfs/gns/literature/Santosh_Tirunagari/GitHub/Unsupervised-Protein-Genes-Diseases-Extraction/Datasets/EU-ADR+GAD
2019-09-10 14:10:45,488 Train: /nfs/gns/literature/Santosh_Tirunagari/GitHub/Unsupervised-Protein-Genes-Diseases-Extraction/Datasets/EU-ADR+GAD/train.csv
2019-09-10 14:10:45,489 Dev: /nfs/gns/literature/Santosh_Tirunagari/GitHub/Unsupervised-Protein-Genes-Diseases-Extraction/Datasets/EU-ADR+GAD/dev.csv
2019-09-10 14:10:45,490 Test: /nfs/gns/literature/Santosh_Tirunagari/GitHub/Unsupervised-Protein-Genes-Diseases-Extraction/Datasets/EU-ADR+GAD/test.csv


/homes/stirunag/anaconda3/envs/pytorch/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated function (or staticmethod) load_classification_corpus. (Use 'flair.datasets' instead.) -- Deprecated since version 0.4.1.
  """Entry point for launching an IPython kernel.
/homes/stirunag/anaconda3/envs/pytorch/lib/python3.7/site-packages/flair/data_fetcher.py:447: DeprecationWarning: Call to deprecated function (or staticmethod) read_text_classification_file. (Use 'flair.datasets' instead.) -- Deprecated since version 0.4.1.
  max_tokens_per_doc=max_tokens_per_doc,
/homes/stirunag/anaconda3/envs/pytorch/lib/python3.7/site-packages/flair/data_fetcher.py:454: DeprecationWarning: Call to deprecated function (or staticmethod) read_text_classification_file. (Use 'flair.datasets' instead.) -- Deprecated since version 0.4.1.
  max_tokens_per_doc=max_tokens_per_doc,
/homes/stirunag/anaconda3/envs/pytorch/lib/python3.7/site-packages/flair/data_fetcher.py:463: Depre

In [18]:
from flair.embeddings import TokenEmbeddings, FlairEmbeddings, WordEmbeddings, PooledFlairEmbeddings, StackedEmbeddings, CharacterEmbeddings, CharLMEmbeddings
from typing import List


# 3. make a list of word embeddings
word_embeddings = [WordEmbeddings('glove'),

                   # comment in flair embeddings for state-of-the-art results
                   FlairEmbeddings('news-forward'),
                   FlairEmbeddings('news-backward'),
                   CharacterEmbeddings(), 
                   PooledFlairEmbeddings('pubmed-backward', pooling='max'),
                   PooledFlairEmbeddings('pubmed-forward', pooling='max')
                   ]



In [19]:
# document_embeddings = DocumentRNNEmbeddings(word_embeddings, hidden_size=512, reproject_words=True, reproject_words_dimension=256)
document_embeddings: DocumentRNNEmbeddings = DocumentRNNEmbeddings(word_embeddings,
                                                                   rnn_type = "LSTM",
                                                                     hidden_size=512,
                                                                     reproject_words=True,
                                                                     reproject_words_dimension=256,
                                                                     )

In [20]:
classifier = TextClassifier(document_embeddings, label_dictionary=corpus.make_label_dictionary(), multi_label=False)



2019-09-10 14:16:06,269 Computing label dictionary. Progress:


100%|██████████| 2688/2688 [00:00<00:00, 396870.22it/s]

2019-09-10 14:16:06,319 [b'PA', b'NA']


In [21]:
trainer = ModelTrainer(classifier, corpus)


In [22]:
# 7. start the training
# trainer.train(data_loc, max_epochs=10)

result_loc = '../flair_models/text_classification/sentiment_model'
trainer.train(result_loc,
              learning_rate=0.15,
              mini_batch_size=128,
              anneal_factor=0.5,
              patience=5,
              max_epochs=150)

2019-09-10 14:18:21,171 ----------------------------------------------------------------------------------------------------
2019-09-10 14:18:21,172 Model: "TextClassifier(
  (document_embeddings): DocumentRNNEmbeddings(
    (embeddings): StackedEmbeddings(
      (list_embedding_0): WordEmbeddings('glove')
      (list_embedding_1): FlairEmbeddings(
        (lm): LanguageModel(
          (drop): Dropout(p=0.05)
          (encoder): Embedding(300, 100)
          (rnn): LSTM(100, 2048)
          (decoder): Linear(in_features=2048, out_features=300, bias=True)
        )
      )
      (list_embedding_2): FlairEmbeddings(
        (lm): LanguageModel(
          (drop): Dropout(p=0.05)
          (encoder): Embedding(300, 100)
          (rnn): LSTM(100, 2048)
          (decoder): Linear(in_features=2048, out_features=300, bias=True)
        )
      )
      (list_embedding_3): CharacterEmbeddings(
        (char_embedding): Embedding(275, 25)
        (char_rnn): LSTM(25, 25, bidirectional=True)
 

2019-09-10 14:23:42,092 epoch 4 - iter 10/21 - loss 0.56578157 - samples/sec: 43.66
2019-09-10 14:23:48,840 epoch 4 - iter 12/21 - loss 0.56816275 - samples/sec: 44.99
2019-09-10 14:23:55,907 epoch 4 - iter 14/21 - loss 0.57008096 - samples/sec: 44.57
2019-09-10 14:24:02,939 epoch 4 - iter 16/21 - loss 0.57287381 - samples/sec: 44.16
2019-09-10 14:24:09,571 epoch 4 - iter 18/21 - loss 0.57620039 - samples/sec: 45.03
2019-09-10 14:24:17,643 epoch 4 - iter 20/21 - loss 0.57667898 - samples/sec: 35.79
2019-09-10 14:24:18,631 ----------------------------------------------------------------------------------------------------
2019-09-10 14:24:18,633 EPOCH 4 done: loss 0.5767 - lr 0.1500
2019-09-10 14:24:21,673 DEV : loss 0.6465857625007629 - score 0.628
2019-09-10 14:24:21,762 BAD EPOCHS (no improvement): 3
2019-09-10 14:24:27,121 ----------------------------------------------------------------------------------------------------
train mode resetting embeddings
train mode resetting embeddin

2019-09-10 14:31:28,741 epoch 10 - iter 4/21 - loss 0.51752160 - samples/sec: 41.96
2019-09-10 14:31:36,597 epoch 10 - iter 6/21 - loss 0.52297181 - samples/sec: 43.54
2019-09-10 14:31:44,464 epoch 10 - iter 8/21 - loss 0.53445375 - samples/sec: 41.87
2019-09-10 14:31:51,302 epoch 10 - iter 10/21 - loss 0.53463703 - samples/sec: 44.50
2019-09-10 14:31:58,196 epoch 10 - iter 12/21 - loss 0.53633061 - samples/sec: 44.01
2019-09-10 14:32:04,860 epoch 10 - iter 14/21 - loss 0.53211868 - samples/sec: 45.74
2019-09-10 14:32:12,538 epoch 10 - iter 16/21 - loss 0.53180880 - samples/sec: 45.89
2019-09-10 14:32:19,104 epoch 10 - iter 18/21 - loss 0.52982923 - samples/sec: 45.66
2019-09-10 14:32:26,962 epoch 10 - iter 20/21 - loss 0.52655568 - samples/sec: 36.66
2019-09-10 14:32:28,256 ----------------------------------------------------------------------------------------------------
2019-09-10 14:32:28,258 EPOCH 10 done: loss 0.5266 - lr 0.1500
2019-09-10 14:32:31,347 DEV : loss 0.6633319854736

2019-09-10 14:39:13,241 ----------------------------------------------------------------------------------------------------
train mode resetting embeddings
train mode resetting embeddings
2019-09-10 14:39:16,509 epoch 16 - iter 0/21 - loss 0.44847989 - samples/sec: 79.83
2019-09-10 14:39:23,270 epoch 16 - iter 2/21 - loss 0.45199121 - samples/sec: 44.05
2019-09-10 14:39:30,065 epoch 16 - iter 4/21 - loss 0.45845220 - samples/sec: 43.26
2019-09-10 14:39:36,786 epoch 16 - iter 6/21 - loss 0.44815680 - samples/sec: 43.86
2019-09-10 14:39:43,470 epoch 16 - iter 8/21 - loss 0.53331277 - samples/sec: 45.35
2019-09-10 14:39:50,143 epoch 16 - iter 10/21 - loss 0.53688136 - samples/sec: 43.59
2019-09-10 14:39:56,714 epoch 16 - iter 12/21 - loss 0.53011861 - samples/sec: 44.08
2019-09-10 14:40:03,715 epoch 16 - iter 14/21 - loss 0.52392573 - samples/sec: 43.88
2019-09-10 14:40:10,057 epoch 16 - iter 16/21 - loss 0.51599856 - samples/sec: 46.62
2019-09-10 14:40:16,908 epoch 16 - iter 18/21 - los

2019-09-10 14:47:04,691 ----------------------------------------------------------------------------------------------------
2019-09-10 14:47:04,693 EPOCH 21 done: loss 0.4829 - lr 0.1500
2019-09-10 14:47:07,835 DEV : loss 0.5192679166793823 - score 0.756
2019-09-10 14:47:07,919 BAD EPOCHS (no improvement): 5
2019-09-10 14:47:07,923 ----------------------------------------------------------------------------------------------------
train mode resetting embeddings
train mode resetting embeddings
2019-09-10 14:47:11,094 epoch 22 - iter 0/21 - loss 0.50028718 - samples/sec: 82.69
2019-09-10 14:47:18,103 epoch 22 - iter 2/21 - loss 0.41668076 - samples/sec: 41.40
2019-09-10 14:47:24,997 epoch 22 - iter 4/21 - loss 0.43426983 - samples/sec: 44.13
2019-09-10 14:47:31,754 epoch 22 - iter 6/21 - loss 0.44265588 - samples/sec: 44.22
2019-09-10 14:47:38,542 epoch 22 - iter 8/21 - loss 0.43878172 - samples/sec: 45.58
2019-09-10 14:47:45,259 epoch 22 - iter 10/21 - loss 0.47374869 - samples/sec: 4

...

2019-09-10 16:58:13,419 epoch 119 - iter 2/21 - loss 0.19706736 - samples/sec: 40.78
2019-09-10 16:58:20,755 epoch 119 - iter 4/21 - loss 0.18024043 - samples/sec: 42.77
2019-09-10 16:58:27,420 epoch 119 - iter 6/21 - loss 0.16716692 - samples/sec: 45.63
2019-09-10 16:58:34,747 epoch 119 - iter 8/21 - loss 0.16659399 - samples/sec: 43.75
2019-09-10 16:58:41,954 epoch 119 - iter 10/21 - loss 0.15218754 - samples/sec: 43.87
2019-09-10 16:58:48,994 epoch 119 - iter 12/21 - loss 0.15603656 - samples/sec: 43.58
2019-09-10 16:58:55,988 epoch 119 - iter 14/21 - loss 0.15877689 - samples/sec: 45.96
2019-09-10 16:59:02,995 epoch 119 - iter 16/21 - loss 0.15387852 - samples/sec: 43.23
2019-09-10 16:59:09,766 epoch 119 - iter 18/21 - loss 0.15555222 - samples/sec: 43.70
2019-09-10 16:59:18,288 epoch 119 - iter 20/21 - loss 0.15570460 - samples/sec: 33.50
2019-09-10 16:59:19,452 ----------------------------------------------------------------------------------------------------
2019-09-10 16:59:19

2019-09-10 17:05:54,452 EPOCH 124 done: loss 0.1542 - lr 0.0006
2019-09-10 17:05:57,626 DEV : loss 0.20120953023433685 - score 0.9345
2019-09-10 17:05:57,709 BAD EPOCHS (no improvement): 3
2019-09-10 17:05:57,713 ----------------------------------------------------------------------------------------------------
train mode resetting embeddings
train mode resetting embeddings
2019-09-10 17:06:00,785 epoch 125 - iter 0/21 - loss 0.14387415 - samples/sec: 85.00
2019-09-10 17:06:08,291 epoch 125 - iter 2/21 - loss 0.13363478 - samples/sec: 43.21
2019-09-10 17:06:15,119 epoch 125 - iter 4/21 - loss 0.15229894 - samples/sec: 43.87
2019-09-10 17:06:21,720 epoch 125 - iter 6/21 - loss 0.16696240 - samples/sec: 44.47
2019-09-10 17:06:28,554 epoch 125 - iter 8/21 - loss 0.15750121 - samples/sec: 44.67
2019-09-10 17:06:35,842 epoch 125 - iter 10/21 - loss 0.15469017 - samples/sec: 42.96
2019-09-10 17:06:42,761 epoch 125 - iter 12/21 - loss 0.14906097 - samples/sec: 43.07
2019-09-10 17:06:49,708 e

2019-09-10 17:13:24,527 epoch 130 - iter 14/21 - loss 0.16441904 - samples/sec: 43.90
2019-09-10 17:13:31,498 epoch 130 - iter 16/21 - loss 0.16152301 - samples/sec: 43.35
2019-09-10 17:13:38,457 epoch 130 - iter 18/21 - loss 0.16188876 - samples/sec: 43.45
2019-09-10 17:13:47,234 epoch 130 - iter 20/21 - loss 0.15478539 - samples/sec: 34.45
2019-09-10 17:13:48,202 ----------------------------------------------------------------------------------------------------
2019-09-10 17:13:48,204 EPOCH 130 done: loss 0.1548 - lr 0.0003
2019-09-10 17:13:51,334 DEV : loss 0.20122718811035156 - score 0.9345
2019-09-10 17:13:51,418 BAD EPOCHS (no improvement): 3
2019-09-10 17:13:51,422 ----------------------------------------------------------------------------------------------------
train mode resetting embeddings
train mode resetting embeddings
2019-09-10 17:13:54,498 epoch 131 - iter 0/21 - loss 0.10394654 - samples/sec: 85.14
2019-09-10 17:14:01,932 epoch 131 - iter 2/21 - loss 0.15178581 - sa

2019-09-10 17:20:36,846 epoch 136 - iter 2/21 - loss 0.18321669 - samples/sec: 41.96
2019-09-10 17:20:43,355 epoch 136 - iter 4/21 - loss 0.17907152 - samples/sec: 44.80
2019-09-10 17:20:50,376 epoch 136 - iter 6/21 - loss 0.17916246 - samples/sec: 42.09
2019-09-10 17:20:57,441 epoch 136 - iter 8/21 - loss 0.16667470 - samples/sec: 42.55
2019-09-10 17:21:03,984 epoch 136 - iter 10/21 - loss 0.16299825 - samples/sec: 45.65
2019-09-10 17:21:10,642 epoch 136 - iter 12/21 - loss 0.17276331 - samples/sec: 43.74
2019-09-10 17:21:17,981 epoch 136 - iter 14/21 - loss 0.16875689 - samples/sec: 44.91
2019-09-10 17:21:24,987 epoch 136 - iter 16/21 - loss 0.16331907 - samples/sec: 42.32
2019-09-10 17:21:31,903 epoch 136 - iter 18/21 - loss 0.15949705 - samples/sec: 42.34
2019-09-10 17:21:39,973 epoch 136 - iter 20/21 - loss 0.15723908 - samples/sec: 35.27
2019-09-10 17:21:41,062 ----------------------------------------------------------------------------------------------------
2019-09-10 17:21:41

{'test_score': 0.9077,
 'dev_score_history': [0.628,
  0.628,
  0.628,
  0.628,
  0.6667,
  0.6875,
  0.6696,
  0.6667,
  0.7054,
  0.6339,
  0.5774,
  0.747,
  0.6786,
  0.628,
  0.7232,
  0.7679,
  0.7351,
  0.628,
  0.628,
  0.6667,
  0.756,
  0.7917,
  0.7024,
  0.7113,
  0.753,
  0.8185,
  0.628,
  0.7857,
  0.756,
  0.7738,
  0.8095,
  0.628,
  0.8036,
  0.8214,
  0.8393,
  0.8571,
  0.6161,
  0.6577,
  0.8274,
  0.8006,
  0.8571,
  0.8571,
  0.8542,
  0.8452,
  0.8571,
  0.8601,
  0.7351,
  0.872,
  0.875,
  0.8601,
  0.8571,
  0.8452,
  0.881,
  0.8899,
  0.8899,
  0.8601,
  0.8958,
  0.8929,
  0.8958,
  0.9018,
  0.8988,
  0.8988,
  0.8869,
  0.8988,
  0.9018,
  0.9018,
  0.9196,
  0.9018,
  0.9226,
  0.9048,
  0.9077,
  0.9226,
  0.8869,
  0.9196,
  0.9286,
  0.9226,
  0.9196,
  0.9196,
  0.9226,
  0.9226,
  0.9256,
  0.9196,
  0.9256,
  0.9226,
  0.9107,
  0.9315,
  0.9286,
  0.9226,
  0.9256,
  0.9226,
  0.9226,
  0.9375,
  0.9315,
  0.9286,
  0.9345,
  0.9226,
  0.9405,
  

In [ ]:
import torch
torch.cuda.is_available()

In [26]:
from flair.models import TextClassifier
from flair.data import Sentence

classifier = TextClassifier.load(result_loc+'/best-model.pt')


2019-09-10 17:34:58,485 loading file ../flair_models/text_classification/sentiment_model/best-model.pt


In [30]:
sentence = Sentence('The FCRL3 polymorphisms associated with rheumatoid arthritis in a Japanese population are not associated per se with rheumatoid arthritis in a Spanish population.')
sentence = Sentence('We obtained evidence from two Caucasian study populations that the His298-allele of PTGES2 Arg298His confers to reduced risk of type 2 diabetes.')
sentence = Sentence('it does not seem that variants in MnSOD, CAT, or GPX1 have an influence on prostate cancer risk in this cohort of men who were smokers or exposed to asbestos, although it is possible that cumulative defects in protection from oxidative stress may result in increased risk of the disease.')
classifier.predict(sentence)
print(sentence.labels)

[NA (0.722224771976471)]


In [ ]:
from tqdm import tqdm

In [ ]:
ground_truth = []
predicted_label = []
for index, rows in tqdm(df.iterrows(), total=df.shape[0]):
    if rows['label'] == 'NA':
       ground_truth.append('Negative')
    else:
       ground_truth.append('Positive')
    sentence = Sentence(rows['text'])
    classifier.predict(sentence)
    predicted_label.append(sentence.labels[0].value)
    
    

In [ ]:
from sklearn.metrics import f1_score, accuracy_score

f1_score(ground_truth, predicted_label, average='micro') 